In [110]:
from notebookutils import mssparkutils
from pyspark.sql.functions import udf, col, from_json, concat_ws, explode, current_timestamp
from pyspark.sql.types import StringType, Row, StructType, StructField, ArrayType, MapType
from pyspark.sql.utils import AnalysisException


from synapse.ml.services import AnalyzeDocument

from delta.tables import *

from synapse.ml.services.openai import OpenAIChatCompletion
import json

StatementMeta(, f6ea8b84-7ff9-4616-bb6a-8da757797b5a, 112, Finished, Available, Finished)

In [111]:
spark.conf.set("spark.databricks.delta.schema.autoMerge.enabled","true")

StatementMeta(, f6ea8b84-7ff9-4616-bb6a-8da757797b5a, 113, Finished, Available, Finished)

In [112]:
# Getting all necessary secrets 

ai_services_key = mssparkutils.credentials.getSecret('https://patsqlkeyvaultnew.vault.azure.net/', 'patsqlDocIntelligenceKey')
ai_services_location = mssparkutils.credentials.getSecret('https://patsqlkeyvaultnew.vault.azure.net/', 'patsqlDocIntelligenceRegion') 
ai_aoai_key = mssparkutils.credentials.getSecret('https://patsqlkeyvaultnew.vault.azure.net/', 'patsqlAOAIKey')
ai_aoai_url = mssparkutils.credentials.getSecret('https://patsqlkeyvaultnew.vault.azure.net/', 'patsqlAOAIURL')

StatementMeta(, f6ea8b84-7ff9-4616-bb6a-8da757797b5a, 114, Finished, Available, Finished)

In [113]:
# Input parameter
document_path = "Files/PDF/MGR65002afs22.pdf"

StatementMeta(, f6ea8b84-7ff9-4616-bb6a-8da757797b5a, 115, Finished, Available, Finished)

In [114]:
df = (
    spark.read.format("binaryFile")
    .load(document_path)
    .limit(10)
    .cache()
)

StatementMeta(, f6ea8b84-7ff9-4616-bb6a-8da757797b5a, 116, Finished, Available, Finished)

In [115]:
analyze_document = (
    AnalyzeDocument()
    .setPrebuiltModelId("prebuilt-layout")
    .setSubscriptionKey(ai_services_key)
    .setLocation(ai_services_location)
    .setImageBytesCol("content")
    .setOutputCol("result")
    .setPages("1-5") # for sake of quick processing, only read the first 15 pages of the documents
)

analyzed_df = (
    analyze_document.transform(df)
    .withColumn("output_content", col("result.analyzeResult.content"))
    .withColumn("paragraphs", col("result.analyzeResult.paragraphs"))).cache()

StatementMeta(, f6ea8b84-7ff9-4616-bb6a-8da757797b5a, 117, Finished, Available, Finished)

In [116]:
analyzed_df = analyzed_df.drop("content")

StatementMeta(, f6ea8b84-7ff9-4616-bb6a-8da757797b5a, 118, Finished, Available, Finished)

In [117]:
# Define the JSON structure you want to extract
json_structure = {
  "grantDetails": {
    "projectReference": "",
    "periodAudited": "",
    "donor": "",
    "pledges": "",
    "countriesConcerned": ""
  },
  "auditType": "",
  "totalValueAudited": {
    "projectedFigures": "",
    "actualFigures": ""
  },
  "expenditureTested": {
    "categories": []
  },
  "auditReportSummary": {
    "opinion": ""
  },
  "keyFinancialFindings": [
    {
      "amount": "",
      "type": ""
    }
  ],
  "recommendations": [],
  "controlWeaknesses": {
    "financialFindings": [
      {
        "nature": "",
        "impact": ""
      }
    ],
    "otherWeaknesses": [
      {
        "nature": "",
        "suggestions": ""
      }
    ]
  },
  "commonControlFindings": [
    {
      "frequency": "",
      "severity": ""
    }
  ]
}

StatementMeta(, f6ea8b84-7ff9-4616-bb6a-8da757797b5a, 119, Finished, Available, Finished)

In [118]:
def make_message(role, content):
    return Row(role=role, content=content, name=role)

StatementMeta(, f6ea8b84-7ff9-4616-bb6a-8da757797b5a, 120, Finished, Available, Finished)

In [119]:
messages = []

for i in analyzed_df.collect(): 
    messages.append(
        [
            (
                [
                    make_message(
                        "system", "You are a useful assistant supporting with structured extraction of information from texts. Don't add any comments or explaining text. Always only return the expected JSON filled with the content that was asked for. When you are asked to extract a Project Reference number search for a string that is between six and 15 characters long and can contain characters and numbers like 'MGR65002', 'PCM031', 'MDRCOVID19', 'M819943'. When an Audittype is asked this can be one of 'ISRS', 'ISA'"
                    ),
                    make_message("user", f"Extract the following information in JSON format: {json.dumps(json_structure)} from the following text: {i['output_content']}"),
                ]
            )
        ]
        )

StatementMeta(, f6ea8b84-7ff9-4616-bb6a-8da757797b5a, 121, Finished, Available, Finished)

In [120]:
colname = ["messages"]
chat_df = spark.createDataFrame(messages, colname)

StatementMeta(, f6ea8b84-7ff9-4616-bb6a-8da757797b5a, 122, Finished, Available, Finished)

In [121]:
# Using a provisioned AOAI gpt-4-32k model in case Fabric Copilot is not available

response = (
    OpenAIChatCompletion()
    .setSubscriptionKey(ai_aoai_key)
    .setDeploymentName("gpt-4-32k")
    .setUrl(ai_aoai_url)
    .setMessagesCol("messages")
    .setErrorCol("error")
    .setOutputCol("chat_completions")
)

StatementMeta(, f6ea8b84-7ff9-4616-bb6a-8da757797b5a, 123, Finished, Available, Finished)

In [122]:
# Using the Fabric built-in AOAI model in case Fabric Copilot is available = no explicit AOAI Model necessary
'''
response =(
    OpenAIChatCompletion()
    .setDeploymentName("gpt-4-32k")
    .setMessagesCol("messages")
    .setErrorCol("error")
    .setOutputCol("chat_completions")
)
'''

StatementMeta(, f6ea8b84-7ff9-4616-bb6a-8da757797b5a, 124, Finished, Available, Finished)

'\nresponse =(\n    OpenAIChatCompletion()\n    .setDeploymentName("gpt-4-32k")\n    .setMessagesCol("messages")\n    .setErrorCol("error")\n    .setOutputCol("chat_completions")\n)\n'

In [123]:
intermediate_df = response.transform(chat_df).select("messages", "chat_completions.choices.message.content")
intermediate_df = intermediate_df.withColumn("content_str", concat_ws("", col("content")))

StatementMeta(, f6ea8b84-7ff9-4616-bb6a-8da757797b5a, 125, Finished, Available, Finished)

In [124]:
myjson_schema = ArrayType(StructType([
    StructField("grantDetails", StructType([
        StructField("projectReference", StringType(), True),
        StructField("periodAudited", StringType(), True),
        StructField("donor", StringType(), True),
        StructField("pledges", StringType(), True),
        StructField("countriesConcerned", StringType(), True)
    ]), True),
    StructField("auditType", StringType(), True),
    StructField("totalValueAudited", StructType([
        StructField("projectedFigures", StringType(), True),
        StructField("actualFigures", StringType(), True)
    ]), True),
    StructField("expenditureTested", StructType([
        StructField("categories", ArrayType(StringType()), True)
    ]), True),
    StructField("auditReportSummary", StructType([
        StructField("opinion", StringType(), True)
    ]), True),
    StructField("keyFinancialFindings", ArrayType(StructType([
        StructField("amount", StringType(), True),
        StructField("type", StringType(), True)
    ])), True),
    StructField("recommendations", ArrayType(StringType()), True),
    StructField("controlWeaknesses", StructType([
        StructField("financialFindings", ArrayType(StructType([
            StructField("nature", StringType(), True),
            StructField("impact", StringType(), True)
        ])), True),
        StructField("otherWeaknesses", ArrayType(StructType([
            StructField("nature", StringType(), True),
            StructField("suggestions", StringType(), True)
        ])), True)
    ]), True),
    StructField("commonControlFindings", ArrayType(StructType([
        StructField("frequency", StringType(), True),
        StructField("severity", StringType(), True)
    ])), True)
])
)

StatementMeta(, f6ea8b84-7ff9-4616-bb6a-8da757797b5a, 126, Finished, Available, Finished)

In [125]:
new_df = intermediate_df.withColumn("parsedContent", from_json(col("content_str"), myjson_schema))

StatementMeta(, f6ea8b84-7ff9-4616-bb6a-8da757797b5a, 127, Finished, Available, Finished)

In [126]:
new_df.cache()

StatementMeta(, f6ea8b84-7ff9-4616-bb6a-8da757797b5a, 128, Finished, Available, Finished)

DataFrame[messages: array<struct<role:string,content:string,name:string>>, content: array<string>, content_str: string, parsedContent: array<struct<grantDetails:struct<projectReference:string,periodAudited:string,donor:string,pledges:string,countriesConcerned:string>,auditType:string,totalValueAudited:struct<projectedFigures:string,actualFigures:string>,expenditureTested:struct<categories:array<string>>,auditReportSummary:struct<opinion:string>,keyFinancialFindings:array<struct<amount:string,type:string>>,recommendations:array<string>,controlWeaknesses:struct<financialFindings:array<struct<nature:string,impact:string>>,otherWeaknesses:array<struct<nature:string,suggestions:string>>>,commonControlFindings:array<struct<frequency:string,severity:string>>>>]

In [127]:
new_df_exploded = new_df.select(explode("parsedContent").alias("parsedContent"))

StatementMeta(, f6ea8b84-7ff9-4616-bb6a-8da757797b5a, 129, Finished, Available, Finished)

In [128]:
new_dfs_info = [
    {"newDataFrameName": "df_grantDetails", "columnNames": ["parsedContent.grantDetails.projectReference", "parsedContent.grantDetails.periodAudited", "parsedContent.grantDetails.donor", "parsedContent.grantDetails.pledges", "parsedContent.grantDetails.countriesConcerned", "parsedContent.auditType", current_timestamp().alias("insert_datetime")]},
    {"newDataFrameName": "df_totalValueAudited", "columnNames": ["parsedContent.grantDetails.projectReference", "parsedContent.totalValueAudited.projectedFigures", "parsedContent.totalValueAudited.actualFigures", current_timestamp().alias("insert_datetime")]},
    {"newDataFrameName": "df_expenditureTested", "columnNames": ["parsedContent.grantDetails.projectReference", "parsedContent.expenditureTested.categories", current_timestamp().alias("insert_datetime")]},
    {"newDataFrameName": "df_auditReportSummary", "columnNames": ["parsedContent.grantDetails.projectReference", "parsedContent.auditReportSummary.opinion", current_timestamp().alias("insert_datetime")]},
    {"newDataFrameName": "df_keyFinancialFindings", "columnNames": ["parsedContent.grantDetails.projectReference", "parsedContent.keyFinancialFindings.amount", "parsedContent.keyFinancialFindings.type", current_timestamp().alias("insert_datetime")]},
    {"newDataFrameName": "df_recommendations", "columnNames": ["parsedContent.grantDetails.projectReference", "parsedContent.recommendations", current_timestamp().alias("insert_datetime")]},
    {"newDataFrameName": "df_controlWeaknessesfinancialFindings", "columnNames": ["parsedContent.grantDetails.projectReference", "parsedContent.controlWeaknesses.financialFindings.nature", "parsedContent.controlWeaknesses.financialFindings.impact", current_timestamp().alias("insert_datetime")]},
    {"newDataFrameName": "df_controlWeaknessesotherWeaknesses", "columnNames": ["parsedContent.grantDetails.projectReference", "parsedContent.controlWeaknesses.otherWeaknesses.nature", "parsedContent.controlWeaknesses.otherWeaknesses.suggestions", current_timestamp().alias("insert_datetime")]},
    {"newDataFrameName": "df_commonControlFindings", "columnNames": ["parsedContent.grantDetails.projectReference", "parsedContent.commonControlFindings.frequency", "parsedContent.commonControlFindings.severity", current_timestamp().alias("insert_datetime")]}
]

StatementMeta(, f6ea8b84-7ff9-4616-bb6a-8da757797b5a, 130, Finished, Available, Finished)

In [129]:
def create_new_dataframes(sourceDataFrame, newDataFrames):
    
    # Dictionary to store the new DataFrames
    new_dfs = {}
    
    # Iterate through the array of newDataFrames
    for row in newDataFrames:
        new_df_name = row["newDataFrameName"]
        column_names = row["columnNames"]
        print(column_names)
        # Select the specified columns from the source DataFrame
        new_df = sourceDataFrame.select(*column_names)
        
        # Store the new DataFrame in the dictionary
        new_dfs[new_df_name] = new_df
    
    return new_dfs

StatementMeta(, f6ea8b84-7ff9-4616-bb6a-8da757797b5a, 131, Finished, Available, Finished)

In [130]:
new_dfs = create_new_dataframes(new_df_exploded, new_dfs_info)

StatementMeta(, f6ea8b84-7ff9-4616-bb6a-8da757797b5a, 132, Finished, Available, Finished)

['parsedContent.grantDetails.projectReference', 'parsedContent.grantDetails.periodAudited', 'parsedContent.grantDetails.donor', 'parsedContent.grantDetails.pledges', 'parsedContent.grantDetails.countriesConcerned', 'parsedContent.auditType', Column<'current_timestamp() AS insert_datetime'>]
['parsedContent.grantDetails.projectReference', 'parsedContent.totalValueAudited.projectedFigures', 'parsedContent.totalValueAudited.actualFigures', Column<'current_timestamp() AS insert_datetime'>]
['parsedContent.grantDetails.projectReference', 'parsedContent.expenditureTested.categories', Column<'current_timestamp() AS insert_datetime'>]
['parsedContent.grantDetails.projectReference', 'parsedContent.auditReportSummary.opinion', Column<'current_timestamp() AS insert_datetime'>]
['parsedContent.grantDetails.projectReference', 'parsedContent.keyFinancialFindings.amount', 'parsedContent.keyFinancialFindings.type', Column<'current_timestamp() AS insert_datetime'>]
['parsedContent.grantDetails.projectR

In [131]:
output_path = 'Tables/'

for df_name, df in new_dfs.items():
        # Write each DataFrame as a Delta Lake table
        df.write \
            .format("delta") \
            .option("mergeSchema", "true") \
            .mode("append") \
            .save(f"{output_path}/{df_name}")


StatementMeta(, f6ea8b84-7ff9-4616-bb6a-8da757797b5a, 133, Finished, Available, Finished)